# 4-24 Demo

Today we'll see a measure of chaotic systems, the *Lyapunov exponents*, and calculate them for orbits in the Lorenz strange attractor. 

The (Lorenz system)[https://en.wikipedia.org/wiki/Lorenz_system] is a 3d system of ODEs 
\begin{align}
\dot{x} &= \sigma(y-x)\\
\dot{y} &= x(\rho - z) - y\\
\dot{z} &= xy - \beta z
\end{align}
which depends on 3 parameters $\sigma, \rho$ and $\beta$. There are various physical systems which this set of equations models - the original example was temperature fluctuations in a fluid layer subject to heating from below and cooling from above, for meteorological applications. 

The most interesting values occur near $\sigma = 10, \rho = 28, \beta = 8/3$. 


In [148]:
# Code, copied from somewhere but straightforward enough, to plot the Lorenz attractor 

x,y,z=var('x,y,z')

# Next we define the parameters
σ=1 # 10
ρ= 15  #28
β=8/3

# The Lorenz equations
lorenz=[σ*(y-x),x*(ρ-z)-y,x*y-β*z]

# Time and initial conditions
N=25000
tmax=250
h=tmax/N
times=srange(0,tmax+h,h)
ics=[0,1,1]
sol=desolve_odeint(lorenz,ics,times,[x,y,z])#,rtol=1e-13,atol=1e-14)
solX = sol[:, 0]
solY = sol[:, 1]
solZ = sol[:, 2]

In [150]:
#line3d(sol)

So we see that there is an attracting fixed point, and the system spirals into this fixed point. As we approach the more interesting values, something **strange** happens! There is a sort of bifurcation that happens but the orbit that shows up is stable but not very regular. 

In [179]:
x,y,z=var('x,y,z')

# Next we define the parameters
σ=10
ρ= 28
β=8/3

# The Lorenz equations
lorenz=[σ*(y-x),x*(ρ-z)-y,x*y-β*z]

# Time and initial conditions
N=25000
tmax=250
h=tmax/N
times=srange(0,tmax+h,h)
ics=[0,1,1]
sol=desolve_odeint(lorenz,ics,times,[x,y,z])#,rtol=1e-13,atol=1e-14)
solX = sol[:, 0]
solY = sol[:, 1]
solZ = sol[:, 2]

In [183]:
#show(line3d(sol, thickness = 0.4))

0.0500000000000000

## Lyapunov exponents

So we see that, despite that fact that this is an *attractor* in the sense that nearby trajectories get pulled in to the trajectories we see, it's not periodic in the sense that it really repeats - like the orbit of the earth around the sun. We'll measure precisely one aspect of this behavior - that the long-term behavior is extremely sensitive to the initial conditions. Exponentially so!

Suppose two trajectories $\overrightarrow{x}_1(t)$ and $\overrightarrow{x}_2(t)$ start out extremely close to eachother, so that at time zero, 
$$|\overrightarrow{x_1}(0) - \overrightarrow{x_2}(0)| = \delta$$  but diverge exponentially, so that 

$$|\overrightarrow{x_1}(t) - \overrightarrow{x_2}(t)| \approx e^{\lambda t} \delta$$

Given $\overrightarrow{x_1}(t)$, the Lyapunov exponents calculate what nearby trajectories $\overrightarrow{x_2}(t)$ have what possible values of $\lambda$. If they are all
1. **Negative** - then nearby trajectories get closer together, and we have a stable attracting trajectory
2. **Positive** - Then nearby trajectories exponentially diverge, our system is highly sensitive on initial conditions, and possibly chaotic, especially if $\overrightarrow{x_1}(t)$ is part of an attractor. 
3. **Zero** - then nearby trajectories can only  diverge or contract sub-exponentially. This usually means they stay around the same distance apart. 

Actually calculating all of the Lyapunov exponents for a dyanmical system is actually not much more difficult that what we will do - but involves some linear algebra. So we will just calculate the exponent $\lambda$ for two nearby trajectories near the interesting locus in the Lorenz system. 

In [146]:
# Let's package this into a function...

def lorenz_solve(initial, N = 25000, tmax = 250, σ=10,ρ= 28, β=8/3):
    x,y,z=var('x,y,z')

    # Next we define the parameters
    σ=10
    ρ= 28
    β=8/3

    # The Lorenz equations
    lorenz=[σ*(y-x),x*(ρ-z)-y,x*y-β*z]

    # Time and initial conditions
    #N=25000
    #tmax=250
    h=tmax/N
    times=srange(0,tmax+h,h)
    ics=initial #[0,1,1]
    sol=desolve_odeint(lorenz,ics,times,[x,y,z])#,rtol=1e-13,atol=1e-14)
    return(sol)


# Basicall all of these trajectories nearby the starting point I gave stay the same for a while ( say, t = 10)  but the diverge

#line3d(lorenz_solve([0,1,1], tmax = 10, N = 2500), color = 'red') + line3d(lorenz_solve([0,1.1,1], tmax = 10, N = 2500), color = 'blue')


#line3d(lorenz_solve([0,1,1], tmax = 21, N = 2500), color = 'red') + line3d(lorenz_solve([0,1.1,1], tmax = 21, N = 2500), color = 'blue')


In [147]:
#However, if we start near the locus where the most "mixing" of the two wings happens, we can see divergence pretty quickly. 

#line3d(lorenz_solve([1,1.5,22], tmax = 3, N = 2500), color = 'red') + line3d(lorenz_solve([1.1,1.5,22], tmax = 3, N = 2500), color = 'blue') + sphere((1,1.5,22),0.5, color = 'green')

We want to calculate the separation for relatively small times between two orbits that start very close to eachother, say 
$$\text{sep}(0)  = \delta \approx 0.0000000001$$
and we want, for say, times $ t\in [0,10]$ for us to have 
$$\text{sep}(t) \approx e^{\lambda t} \delta$$. 

So we just take a logarithmic plot of the separation and its slope will be our estimated value for $\lambda$, since we have 

$$\log(\text{sep}(t)) \approx \lambda t + \log \delta $$

In [143]:
# Of course the separation between two points is calculated by the Pythagorean theorem 

def sep(x1, x2):
    return sqrt((x1[0] - x2[0])^2 + (x1[1] - x2[1] )^2 + (x1[2] - x2[2])^2)

# Setting some useful values
tmax = 10
N = 1000
h=tmax/N
δ = 1e-9
#
ic1 = [1,1.5,22]
ic2 = [1 + δ, 1.5, 22]
sol1 = lorenz_solve(ic1, tmax = tmax, N = N)
sol2 = lorenz_solve(ic2, tmax = tmax, N = N)
seps = [sep(sol1[i], sol2[i]) for i in range(len(sol1))]
points = [(h*i, seps[i]) for i in range(len(sol1))]
#list_plot(points, plotjoined = True)

#logpoints = [(h*i, log(seps[i])) for i in range(len(sol1)) if h*i < 8]
#list_plot(logpoints, plotjoined = True)

In [144]:
# finding the approximate slope

#λ,t,c = var('λ,t, c')
#model(t) = λ*t + c
#fit = find_fit(logpoints, model, solution_dict = True)
#fit[λ]
#list_plot(logpoints, plotjoined = True) + plot(fit[λ]*t + fit[c], (t,0,8), color = 'red')